In [ ]:
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD

Using TensorFlow backend.


In [ ]:
#function to load and pre-process data set
def load_dataset():
	#loading data 
	(trainX, trainY), (testX, testY) = mnist.load_data()
	#re-shaping data to a single channel
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
	#encoding target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

In [ ]:
#function is used for data normalisation and pixel conversion
def prep_pixels(train, test):
	#data casting to float values
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	#normalisation to a range from 0 to 1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	#function returns normalised image
	return train_norm, test_norm

In [ ]:
#fuction to define CNN model
def define_model():
	model = Sequential()
	
	#adding convolutional layer with 3x3 filters and input_shape = 28x28 in gray-scale
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	#adding max pooling of 2x2 
	model.add(MaxPooling2D((2, 2)))
 
	#input layer
	model.add(Flatten())
	#hidden layer
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	#output layer
	model.add(Dense(10, activation='softmax'))
 
	#compile model
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

##Preprocessing the images

In [ ]:
#function implements cross-validation, model fitting and score evaluation 
def evaluate_model(dataX, dataY, n_folds=5):
	scores, histories = list(), list()
	#cross-validation prepartion
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	#split enumeration
	for train_ix, test_ix in kfold.split(dataX):
		#model definition
		model = define_model()

		trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
		#model fitting
		history = model.fit(trainX, trainY, epochs=1, batch_size=32, validation_data=(testX, testY), verbose=0)
		
		#model evaluation
		_, acc = model.evaluate(testX, testY, verbose=0)
		print('> %.3f' % (acc * 100.0))
		
		#storing scores
		scores.append(acc)
		histories.append(history)
	return scores, histories, model


In [ ]:
trainX, trainY, testX, testY = load_dataset()

11493376/11490434 [==============================] - 1s 0us/step


In [ ]:
trainX, testX = prep_pixels(trainX, testX)

In [ ]:
scores, histories, model = evaluate_model(trainX, trainY)







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








> 97.600
> 96.825
> 97.458
> 97.592
> 97.258


In [ ]:
	model.save('final_model.h5')

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

In [ ]:
#function to load image from file
def load_image(filename):
	
	#image loading
	img = load_img(filename, grayscale=True, target_size=(28, 28))
	
	#image conversion to array
	img = img_to_array(img)
	#re-shaping data to a single channel
	img = img.reshape(1, 28, 28, 1)
	
	#preparing pixel data
	img = img.astype('float32')
	img = img / 255.0
	return img

In [ ]:
def run_example():
	
	#image amd model loading
	img = load_image('/content/sample_data/sample_image.png')
	model = load_model('final_model.h5')
	
	#class prediction
	digit = model.predict_classes(img)
	print(digit[0])

In [ ]:
run_example()

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:104: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


7
